In [9]:
import numpy as np
import pandas as pd
import matplotlib as mpl

# Wymuś interaktywny backend (okno) – działa na większości Linuxów
# Jeśli nie masz Qt, zmień na "TkAgg"
mpl.use("QtAgg")

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from pathlib import Path
from zipfile import Path as ZipPath


# =========================
# USTAWIENIA
# =========================
ZIP = Path("/home/piegus/ncbj/Inzynierka/python/Dopasowanie.zip")

INTERVALS = {
    "zrzut01": ("2025-12-08 19:55:30", "2025-12-08 20:03:30"),
}

CUTS = {
    "zrzut01": ("2025-12-08 19:57:30", "2025-12-08 20:01:30"),
}


# =========================
# 1) ZIP -> lista CSV
# =========================
zip_root = ZipPath(ZIP)
fps = {p.stem: p for p in zip_root.iterdir() if p.suffix.lower() == ".csv"}

print("Pliki w ZIP:")
for k, v in fps.items():
    print(f"  {k:20s} exists={v.exists()}")

required = ["IRL-2.1", "IRL-2.2", "trpp"]
missing = [k for k in required if k not in fps]
if missing:
    raise FileNotFoundError(f"Brakuje w ZIP: {missing}. Dostępne: {list(fps.keys())}")


# =========================
# 2) Wczytanie CSV -> dfs
# =========================
dfs = {}
for k, p in fps.items():
    with p.open("r") as f:
        dfs[k] = (
            pd.read_csv(f, parse_dates=["UTC"], index_col="UTC")
            .sort_index()
        )


# =========================
# 3) Sklejenie do df
# =========================
df = pd.concat(
    [
        dfs["IRL-2.1"][["Flux, cps"]].rename(columns={"Flux, cps": "IRL-2.1"}),
        dfs["IRL-2.2"][["Flux, cps"]].rename(columns={"Flux, cps": "IRL-2.2"}),
        dfs["trpp"],
    ],
    axis=1,
).sort_index()

print("\nKolumny df:", list(df.columns))

# Bezpiecznik na brak kolumn TRPP
need_trpp = ["PAR", "PK2", "PK6", "PB1"]
missing_trpp = [c for c in need_trpp if c not in df.columns]
if missing_trpp:
    print("UWAGA: brakuje kolumn TRPP:", missing_trpp)
    # nie wywalam błędu, tylko nie będziemy ich rysować
    need_trpp = [c for c in need_trpp if c in df.columns]


# =========================
# FUNKCJE RYSUJĄCE
# =========================
def plot_full(df: pd.DataFrame) -> None:
    fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(12, 12), sharex=True)

    df.plot(ax=ax1, y=["IRL-2.1", "IRL-2.2"], logy=True, grid=True, marker=".")
    if need_trpp:
        df.plot(ax=ax2, y=need_trpp, grid=True, marker=".")

    ax1.set_ylabel("Flux, cps")
    ax2.set_ylabel("TRPP")
    ax2.set_xlabel("UTC")

    plt.tight_layout()
    plt.show()


def plot_interval(df: pd.DataFrame, name: str, t0: str, t1: str) -> None:
    df_cut = df.loc[t0:t1]
    if df_cut.empty:
        print(f"{name}: pusty zakres")
        return

    fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(16, 12), sharex=True)

    df_cut.plot(ax=ax1, y=["IRL-2.2"], logy=True, grid=True, marker=".")
    if "PK6" in df_cut.columns:
        df_cut.plot(ax=ax2, y=["PK6"], grid=True, marker=".")

    ax1.set_title(f"{name} | {t0} → {t1}")
    ax2.set_xlabel("UTC")

    ax2.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M:%S"))
    ax2.xaxis.set_major_locator(mdates.AutoDateLocator())

    plt.tight_layout()
    plt.show()


def plot_cut_relative_time(df: pd.DataFrame, name: str, t0: str, t1: str) -> None:
    maly_df = df.loc[t0:t1].copy()
    if maly_df.empty:
        print(f"{name}: pusty zakres")
        return

    t = (maly_df.index - maly_df.index[0]).total_seconds().to_numpy()
    y = maly_df["IRL-2.2"].to_numpy(dtype=float)

    # sanity-check
    print(
        f"\n{name}: N={len(t)}, t=[{t[0]}, {t[-1]}] s, "
        f"y_min={np.nanmin(y)}, y_max={np.nanmax(y)}"
    )

    fig = plt.figure(figsize=(10, 5))
    plt.plot(t, y, ".", markersize=4)
    plt.yscale("log")
    plt.grid(True)
    plt.xlabel("t, s")
    plt.ylabel("IRL-2.2, cps")
    plt.title(f"{name}: IRL-2.2 vs t")
    plt.tight_layout()
    plt.show()


# =========================
# URUCHOMIENIE
# =========================
plot_full(df)

for name, (t0, t1) in INTERVALS.items():
    plot_interval(df, name, t0, t1)

for name, (t0, t1) in CUTS.items():
    plot_cut_relative_time(df, name, t0, t1)



Pliki w ZIP:
  IRL-2.2              exists=True
  IRL-2.1              exists=True
  trpp                 exists=True

Kolumny df: ['IRL-2.1', 'IRL-2.2', 'PLC_time', 'PAR', 'PK1', 'PK2', 'PK3', 'PK4', 'PK5', 'PK6', 'PB1', 'PB2', 'PB3', 'PB4', 'PB5', 'PB6']


/tmp/ipykernel_15853/4225311941.py:95: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()
/tmp/ipykernel_15853/4225311941.py:117: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()



zrzut01: N=5053, t=[0.0, 240.930109978] s, y_min=16967.4296875, y_max=54392.03125


/tmp/ipykernel_15853/4225311941.py:143: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()


In [10]:
import pandas as pd
from zipfile import Path as ZipPath
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
import pandas as pd
	
import matplotlib as mpl
mpl.use("pgf")

mpl.rcParams.update({
    "pgf.texsystem": "lualatex",
    "text.usetex": True,
    "font.family": "serif",
})
import matplotlib.pyplot as plt
fps = {x.stem: x for x in ZipPath('Dopasowanie.zip', at='/').iterdir()}
fps   
for k,v in fps.items():
    print(k, v.exists())  
    %%time

dfs = {}
for k,v in fps.items():
    with v.open() as f:
        dfs[k] = pd.read_csv(f, parse_dates=['UTC'], index_col='UTC')  
        df = pd.concat(
    [
        dfs["IRL-2.1"][["Flux, cps"]].rename(columns={"Flux, cps": "IRL-2.1"}),
        dfs["IRL-2.2"][["Flux, cps"]].rename(columns={"Flux, cps": "IRL-2.2"}),
        dfs["trpp"]
    ],
    axis=1
).sort_index()

fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(12,12), sharex=True)

# GÓRA: IRL (tylko raz)
df.plot(
    ax=ax1,
    y=["IRL-2.1", "IRL-2.2"],
    logy=True,
    grid=True,
    marker=".",
)

# DÓŁ: TRPP
df.plot(
    ax=ax2,
    y=["PAR", "PK2", "PK6", "PB1"],
    grid=True,
    marker=".",
)

ax1.legend()
ax2.legend()
plt.tight_layout()
plt.show()
                intervals = {,
    "zrzut01": slice("2025-12-08 19:55:30", "2025-12-08 20:03:30"),
   
}

[(k, v.start, v.stop) for k, v in intervals.items()]            
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

for name, sl in intervals.items():

    df_cut = df.loc[sl]
    if name == "zrzuty":
        continue
    if df_cut.empty:
        print(f"{name}: pusty zakres")
        continue

    fig, (ax1, ax2) = plt.subplots(
        nrows=2,
        figsize=(16, 12),
        sharex=True
    )

    # ===== GÓRA: IRL =====
    df_cut.plot(
        ax=ax1,
        y=["IRL-2.2"],
        logy=True,
        grid=True,
        marker=".",
    )

    # ===== DÓŁ: PK6 =====
    df_cut.plot(
        ax=ax2,
        y=["PK6"],
        grid=True,
        marker=".",
    )

    # format osi X
    ax1.set_title(f"{name} | {sl.start} → {sl.stop}")
    ax2.set_xlabel("UTC")

    plt.tight_layout()
    plt.show()
df_cut = df.loc[intervals["zrzut01"]]

fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(12,12), sharex=True)

df_cut.plot(
    ax=ax1,
    y=["IRL-2.2"],
    logy=True,
    grid=True,
    marker=".",
)

df_cut.plot(
    ax=ax2,
    y=["PK6"],
    grid=True,
    marker=".",
)

plt.tight_layout()
plt.show()
df_cut = df.loc[intervals["zrzut01"]]

fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(12,12), sharex=True)

df_cut.plot(
    ax=ax1,
    y=["IRL-2.2"],
    logy=True,
    grid=True,
    marker=".",
)

df_cut.plot(
    ax=ax2,
    y=["PK6"],
    grid=True,
    marker=".",
)

plt.tight_layout()
plt.show()
cuts = {
    "zrzut01": ("2025-12-08 19:57:30", "2025-12-08 20:01:30"),
   
}

for name, (t0, t1) in cuts.items():
    maly_df = df.loc[t0:t1]

    t = (maly_df.index - maly_df.index[0]).total_seconds()
    y = maly_df["IRL-2.2"]

IndentationError: unexpected indent (2588695334.py, line 61)

# wczytywanie bezpośrednio z archiwum

In [ ]:
fps = {x.stem: x for x in ZipPath('Dopasowanie.zip', at='/').iterdir()}
fps                                                                

{'IRL-2.2': Path('Dopasowanie.zip', 'IRL-2.2.csv'),
 'IRL-2.1': Path('Dopasowanie.zip', 'IRL-2.1.csv'),
 'trpp': Path('Dopasowanie.zip', 'trpp.csv')}

In [ ]:
for k,v in fps.items():
    print(k, v.exists())

IRL-2.2 True
IRL-2.1 True
trpp True


In [ ]:
%%time

dfs = {}
for k,v in fps.items():
    with v.open() as f:
        dfs[k] = pd.read_csv(f, parse_dates=['UTC'], index_col='UTC')


CPU times: user 1.12 s, sys: 58.6 ms, total: 1.18 s
Wall time: 1.22 s


# połączenie w jednego dataframe


In [ ]:
df = pd.concat(
    [
        dfs["IRL-2.1"][["Flux, cps"]].rename(columns={"Flux, cps": "IRL-2.1"}),
        dfs["IRL-2.2"][["Flux, cps"]].rename(columns={"Flux, cps": "IRL-2.2"}),
        dfs["trpp"]
    ],
    axis=1
).sort_index()


In [ ]:
print(df.columns.tolist())
print(dfs.keys())

['IRL-2.1', 'IRL-2.2', 'PLC_time', 'PAR', 'PK1', 'PK2', 'PK3', 'PK4', 'PK5', 'PK6', 'PB1', 'PB2', 'PB3', 'PB4', 'PB5', 'PB6']
dict_keys(['IRL-2.2', 'IRL-2.1', 'trpp'])


In [ ]:

fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(12,12), sharex=True)

# GÓRA: IRL (tylko raz)
df.plot(
    ax=ax1,
    y=["IRL-2.1", "IRL-2.2"],
    logy=True,
    grid=True,
    marker=".",
)

# DÓŁ: TRPP
df.plot(
    ax=ax2,
    y=["PAR", "PK2", "PK6", "PB1"],
    grid=True,
    marker=".",
)

ax1.legend()
ax2.legend()
plt.tight_layout()
plt.show()

/tmp/ipykernel_15853/1301507304.py:23: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()


In [ ]:
intervals = {,
    "zrzut01": slice("2025-12-08 19:55:30", "2025-12-08 20:03:30"),
   
}

[(k, v.start, v.stop) for k, v in intervals.items()]
    

SyntaxError: invalid syntax (4106393206.py, line 1)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

for name, sl in intervals.items():

    df_cut = df.loc[sl]
    if name == "zrzuty":
        continue
    if df_cut.empty:
        print(f"{name}: pusty zakres")
        continue

    fig, (ax1, ax2) = plt.subplots(
        nrows=2,
        figsize=(16, 12),
        sharex=True
    )

    # ===== GÓRA: IRL =====
    df_cut.plot(
        ax=ax1,
        y=["IRL-2.2"],
        logy=True,
        grid=True,
        marker=".",
    )

    # ===== DÓŁ: PK6 =====
    df_cut.plot(
        ax=ax2,
        y=["PK6"],
        grid=True,
        marker=".",
    )

    # format osi X
    ax1.set_title(f"{name} | {sl.start} → {sl.stop}")
    ax2.set_xlabel("UTC")

    plt.tight_layout()
    plt.show()



/tmp/ipykernel_5373/844112070.py:41: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()
/tmp/ipykernel_5373/844112070.py:41: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()
/tmp/ipykernel_5373/844112070.py:41: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()
/tmp/ipykernel_5373/844112070.py:41: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()
/tmp/ipykernel_5373/844112070.py:41: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()
/tmp/ipykernel_5373/844112070.py:41: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()
/tmp/ipykernel_5373/844112070.py:41: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()
/tmp/ipykernel_5373/844112070.py:41: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()
/tmp/ipy

In [ ]:
df_cut = df.loc[intervals["zrzut01"]]

fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(12,12), sharex=True)

df_cut.plot(
    ax=ax1,
    y=["IRL-2.2"],
    logy=True,
    grid=True,
    marker=".",
)

df_cut.plot(
    ax=ax2,
    y=["PK6"],
    grid=True,
    marker=".",
)

plt.tight_layout()
plt.show()


/tmp/ipykernel_5373/400118637.py:22: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()


In [ ]:
df_cut = df.loc[intervals["zrzut01"]]

fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(12,12), sharex=True)

df_cut.plot(
    ax=ax1,
    y=["IRL-2.2"],
    logy=True,
    grid=True,
    marker=".",
)

df_cut.plot(
    ax=ax2,
    y=["PK6"],
    grid=True,
    marker=".",
)

plt.tight_layout()
plt.show()


/tmp/ipykernel_5373/4119963235.py:21: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()


In [ ]:
cuts = {
    "zrzut01": ("2025-12-08 19:57:30", "2025-12-08 20:01:30"),
   
}

for name, (t0, t1) in cuts.items():
    maly_df = df.loc[t0:t1]

    t = (maly_df.index - maly_df.index[0]).total_seconds()
    y = maly_df["IRL-2.2"]

In [ ]:
t

Index([         0.0,  0.023238897,  0.099303007,  0.119539976,  0.199012995,
        0.220527888,  0.299893856,  0.320500851,  0.400672913,  0.423434973,
       ...
       35.500520945, 35.521129847, 35.600239039,  35.62114501,  35.70023799,
       35.720597983, 35.799998045, 35.820545912, 35.899873972, 35.922618866],
      dtype='float64', name='UTC', length=754)

In [ ]:

# def lin_fit(t,x1,x2,a,b,c,T,x):   # funkcja do segmentu liniowego
#     y = a*t+b
#     return y

# def par_fit(t,x1,x2,a,b,c,T,x):    # funkcja do segmentu paraboli
#     d=-2*c*x1
#     f = c*x2**2 + d*x2
#     y = c*t**2 *d*t + f
#     return y

# # def exp_fit(t,x1,x2,a,b,c,T,x):   # funkcja do segmentu exp
# #     d=-2*c*x1
# #     f = c*x2**2 + d*x2
# #     g = (c*x2**2 + d*x2+f)/np.exp(x2/T)
# #     y = np.exp(t/T)*g
# #     return y

# def exp_fit(t, a, T):   # funkcja do segmentu exp
#     y = np.exp(t/T) *a
#     return y



      zrzut   T_hat [s]  sigma_T [s]       rho  delta_rho
0   zrzut01  212.178116     0.080149  0.051887   0.000017
1   zrzut02   76.599384     0.120679  0.118497   0.000139
2   zrzut03   61.954112     0.105184  0.138256   0.000167
3   zrzut04   76.469017     0.137211  0.118647   0.000158
4   zrzut05   63.597868     0.150027  0.135699   0.000229
5   zrzut06   63.302709     0.127898  0.136151   0.000196
6   zrzut07   71.933919     0.148625  0.124122   0.000188
7   zrzut08   76.516696     0.123147  0.118592   0.000142
8   zrzut09   71.661953     0.126130  0.124467   0.000161
9   zrzut10   67.595633     0.155262  0.129877   0.000216
10  zrzut11   67.910129     0.155538  0.129441   0.000215
      zrzut           T ± ΔT               ρ ± Δρ
0   zrzut01  212.178 ± 0.080  0.051887 ± 0.000017
1   zrzut02   76.599 ± 0.121  0.118497 ± 0.000139
2   zrzut03   61.954 ± 0.105  0.138256 ± 0.000167
3   zrzut04   76.469 ± 0.137  0.118647 ± 0.000158
4   zrzut05   63.598 ± 0.150  0.135699 ± 0.000229
5   

Funkcja: scipy.optimize.curve_fit – nieliniowe MNK (Levenberg–Marquardt domyślnie).
exp_fit – Twoja funkcja modelu 
f(t; a,T) zwracająca wartościdopasowania dla wektora t
t – wektor/seria argumentu niezależnego (np. czas).
y – wektor danych obserwowanych (zmienna zależna).
p0=[y.iloc[0], 60] – zgad startowy parametrów [alfa i T ]
maxfev=10000 – maks. liczba wywołań funkcji modelu (zwiększa szansę zbieżności).

(array([26383.68778465,    67.91012883]), array([20.79476392,  0.15553792]))

popt – wektor estymowanych parametrów
pcov – przybliżona macierz kowariancji estymatorów (Podaje nam wartość z moich danych odchylenia standardowego, ale zwraca jest jako simga^2 więc trzeba ją wstaić pod pierwiastek drugiego stopnia )

In [ ]:
           # a, T

a = 26383.7 ± 21
T = 67.9101 ± 0.16 [h]


In [ ]:
# plt.figure(figsize=(20,12))
# plt.plot(t, y, marker='.', label=cn)
# plt.plot(t, exp_fit(t, *popt), 'r-', label='fit')
# plt.axvline(x= popt[0], color='gray', linestyle='--')
# plt.axvline(x= popt[1], color='gray', linestyle=':')
# plt.grid(True, alpha=0.3)
# plt.legend()
# plt.xlabel('Czas, s')
# plt.ylabel('Liczba zliczen, cps')
# plt.tight_layout()
# plt.show()
# # (opcjonalnie) szybka diagnostyka dopasowania
# res = y - exp_fit(t, *popt)
# print("Parametry (x1, x2_raw, a, b, p, c, T) =", [f"{v:.3g}" for v in popt])
# print(f"RMSE = {np.sqrt(np.mean(res**2)):.6g}, MAE = {np.mean(np.abs(res)):.6g}")


Numer telefonu do tomka

In [ ]:
# Bohater
# 791567006

In [ ]:
# def rho_T(T_hat, L,B, b_i, lambda_i):
#     suma = np.sum((b_i * B) / (1 + T_hat/lambda_i))
#     nie_suma = L/(T_hat*B)
#     rho = nie_suma + suma
#     return rho
# def sigma_rho_T(sigma_T, L,B, b_i, lambda_i, T_hat):
#     suma = np.sum((b_i * lambda_i) / (B * (lambda_i*T_hat + 1)**2))
#     nie_suma = L/((T_hat**2)*B)
#     sigma_rho = (nie_suma + suma)*sigma_T
#     return sigma_rho
def rho_T(T_hat, L,B, b_i, lambda_i):
    suma = np.sum((b_i) / ((1 /T_hat)+lambda_i))
    nie_suma = 1/(T_hat*B)
    rho = nie_suma * (L+ suma)
    return rho
def sigma_rho_T(sigma_T, L,B, b_i, lambda_i, T_hat):
    suma = np.sum((b_i * lambda_i) / (B * (lambda_i*T_hat + 1)**2))
    nie_suma = L/((T_hat**2)*B)
    sigma_rho = (nie_suma + suma)*sigma_T
    return sigma_rho

In [ ]:
# prosta funkcja "±" jak w Twoim użyciu pm()
def pm(val, err, vfmt="{:.2f}", efmt="{:.2f}"):
    return f"{vfmt.format(val)} ± {efmt.format(err)}"

ρ(T) = 0.12944096216464201
Δρ(T) = 0.00021483747091936189
ρ ± Δρ = 0.129441 ± 0.000215
T ± ΔT = 67.91 ± 0.16 [s]
